In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from IPython.core.display import display, HTML
from string import Template
import pandas as pd
import json, random


In [3]:
# Get the D3 host locally. 
HTML('<script src="https://d3js.org/d3.v3.js"></script>')

In [4]:
#HTML templet
html_template = Template('''
<style> $css_text </style>
<div id="graph-div"></div>
<script> $js_text </script>
''')

In [5]:
# Css templet
css_text = '''

.bar {
  fill: steelblue;
}

.bar:hover {
  fill: brown;
}

.axis {
  font: 10px sans-serif;
}

.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.x.axis path {
  display: none;
}

.legenda {
  border: black;
  border-width: 1px;
  font-family: verdana;
  font-size: 14 px;
}

'''

In [6]:
#Java script templet
js_text_template = Template('''

var margin = {top: 20, right: 20, bottom: 30, left: 40},
    width = 500 - margin.left - margin.right,
    height = 300 - margin.top - margin.bottom;

var x = d3.scale.ordinal()
    .rangeRoundBands([0, width], .1);

var y = d3.scale.linear()
    .range([height, 0]);

var xAxis = d3.svg.axis()
    .scale(x)
    .orient("bottom");

var yAxis = d3.svg.axis()
    .scale(y)
    .orient("left");

var svg = d3.select("#graph-div").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

var data = $data ;

  x.domain(data.map(function(d) { return d.letter; }));
  y.domain([0, d3.max(data, function(d) { return d.y; })]);

  svg.append("g")
      .attr("class", "x axis")
      .attr("transform", "translate(0," + height + ")")
      .call(xAxis);

  svg.append("g")
      .attr("class", "y axis")
      .call(yAxis);

  svg.selectAll(".bar")
      .data(data)
    .enter().append("rect")
      .attr("class", "bar")
      .attr("x", function(d) { return x(d.letter); })
      .attr("width", x.rangeBand())
      .attr("y", function(d) { return y(d.y); })
      .attr("height", function(d) { return height - y(d.y); });

''')

In [7]:
js_text_template_2 = Template('''

var bars = svg.selectAll(".bar").data($data);
  
bars
  .transition()
    .attr("y", function(d) { return y(d.y); })
    .attr("height", function(d) { return height - y(d.y); });

''')

In [8]:
data = pd.DataFrame({'letter': ['A','B','C','D'], 'y': [1,1,1,1]})
data.head()

,letter,y
0,A,1
1,B,1
2,C,1
3,D,1


In [9]:
jdata = json.dumps(data.to_dict(orient='records'))
js_text = js_text_template.substitute({'data': jdata })
HTML(html_template.substitute({'css_text': css_text, 'js_text': js_text}))

In [10]:
data['y'] = [random.uniform(0,1) for d in data['y']]
js_text = js_text_template_2.substitute({'data': json.dumps(data.to_dict(orient='records'))})
res = f'<script>{js_text}</script>'
HTML(res)

In [11]:
HTML("""<script src="https://d3js.org/d3.v3.min.js"></script>""")

In [12]:
# Css templet
css_text_h = '''

.bar {
  fill: steelblue;
}

.bar:hover {
  fill: brown;
}

.axis {
  font: 10px sans-serif;
}

.axis path,
.axis line {
  fill: none;
  stroke: #000;
  shape-rendering: crispEdges;
}

.x.axis path {
  display: none;
}

.legenda {
  border: black;
  border-width: 1px;
  font-family: verdana;
  font-size: 14 px;
}

'''

In [13]:
huygens_template = Template("""

function fontit(group) {
  if (group == 1) {
      return 'italic'
  }

  else if (group == 2) {
     return 'normal'
  }

  else {

    return 'oblique'

  }
}




function groupx(n) {
  var offset = 250
  if (n.group == 1) { return n.x - offset }
  else if (n.group == 2) { return n.x + offset}
  else {return n.x}

  }


var margin = {top: 20, right: 20, bottom: 30, left: 40},
    width = 500 - margin.left - margin.right,
    height = 300 - margin.top - margin.bottom;


var svg = d3.select("this").append("svg")
    .attr("width", width + margin.left + margin.right)
    .attr("height", height + margin.top + margin.bottom)
  .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");


var domainOnlyScale = d3.scale.linear().domain([1,10, 30, 80, 500, 1300])
                               .range([6,10,14,18,24]) //.rangeRoundBands([2,36])
                               .nice()
                               .clamp(true)
                                ;


var color = d3.scale.ordinal(d3.schemeCategory10);

var attractForce = d3.forceManyBody().strength(-800).distanceMax(1200).distanceMin(400);


var simulation = d3.forceSimulation()
    .force("link", d3.forceLink().id(function(d) { return d.id; }))
    .force("charge", attractForce)
    .force("center", d3.forceCenter(width / 2, height / 2));

d3.json($jsonsrc, function(error, graph) {
  if (error) throw error;

  var link = svg.append("g")
    .attr("class", "links")
    .selectAll("line")
    .data(graph.links)
    .enter().append("line")
      .attr("stroke-width", "1px")
      .style("stroke", "#ccc");

  var node = svg.append("g")
    .attr("class", "nodes")
    .selectAll("g")
    .data(graph.nodes)
    .enter().append("g")

  var circles = node.append("path")
     .attr("d", d3.symbol()
        .size(100)
        .type( function(d) { if
          (d.group == 1) { return d3.symbolCross; } else if
          (d.group == 2) { return d3.symbolSquare; } else
          { return d3.symbolDiamond;}
        }))
      .attr("fill", function(d) { return color(d.group); })
      .style("stroke", "black")
      .style("stroke-width", "1px")
      .call(d3.drag()
          .on("start", dragstarted)
          .on("drag", dragged)
          .on("end", dragended));

  var lables = node.append("text")
      .text(function(d) {
        return d.id;
      })
      .attr("font-family", "verdana")
      .attr("font-size", function(d) {return domainOnlyScale(d.w) + 'pt'})
      .attr("font-style", function(d) {return fontit(d.group)} )
      .style("fill", function(d) { return color(d.group); })
      .style("text-anchor", "middle")
      .attr('y', 3);

  node.append("title")
      .text(function(d) { return d.id; });

  simulation
      .nodes(graph.nodes)
      .on("tick", ticked);

  simulation.force("link")
      .links(graph.links);

  function ticked() {


    link
        .attr("x1", function(d) { return d.source.x; })
        .attr("y1", function(d) { return d.source.y; })
        .attr("x2", function(d) { return groupx(d.target); })
        .attr("y2", function(d) { return d.target.y; });

    node
        .attr("transform", function(d) {
          return "translate(" + groupx(d) + "," + d.y  + ")";
        })

  }
})



function dragstarted(d) {
  if (!d3.event.active) simulation.alphaTarget(0.3).restart();
  d.fx = d.x;
  d.fy = d.y;
}

function dragged(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

function dragended(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  d.fx = null;
  d.fy = null;
}
""")

In [16]:
js_text = huygens_template.substitute({'jsonsrc': srcfl })
html_template2 = Template('''
<style> $css_text </style>
<div id="graph-div2"></div>
<script> $js_text </script>
''')
HTML(html_template2.substitute(css_text=css_text_h, js_text=js_text))

In [17]:
with open('js/samengesteld2.json') as jsonfile:
    srcfl = jsonfile.read()

In [18]:
type(srcfl)

str

In [19]:
js_text = huygens_template.substitute({'jsonsrc': srcfl})
HTML('<script src="https://d3js.org/d3.v3.min.js">' + js_text + '</script>')

In [20]:
from IPython.display import IFrame
display(IFrame(width="100%", height="504",
  src="https://observablehq.com/embed/@hoekr/group-in-a-box-layout?cells=chart%2Cstyle"))

In [21]:
HTML("<script> "+ giab + "</script>")

NameError: name 'giab' is not defined

In [22]:
import sys
!{sys.executable} -m pip install observable_jupyter

Defaulting to user installation because normal site-packages is not writeable


In [23]:
from observable_jupyter import embed
embed('@mbostock/epicyclic-gearing', cells=['graphic'], inputs={'speed': 0.4})

In [24]:
embed(
    '@observablehq/visualize-a-data-frame-with-observable-in-jupyter',
    cells=['vegaPetalsWidget', 'viewof sepalLengthLimits', 'viewof sepalWidthLimits'],
)


In [25]:
import json
with open('js/jsongraph.json') as infl:
    jsondata = json.load(infl)

In [26]:
jsondata
    

{'directed': False,
 'multigraph': False,
 'graph': {},
 'nodes': [{'id': 0, 'name': 'RCE', 'category': 'u'},
  {'id': 1,
   'name': 'Groenman, Sj.',
   'organisation': 'REMP',
   'period_start': '1952',
   'last_known_date': '1969',
   'norm_name': 'Groenman, Sj.',
   'prs_country': 'NL',
   'orgcat': 3,
   'category': 'u'},
  {'id': 2,
   'name': 'Hofstede, B.P.',
   'organisation': 'Dutch Government',
   'period_start': '',
   'last_known_date': '',
   'norm_name': 'Hofstede, B.P.',
   'prs_country': 'NL',
   'orgcat': 3,
   'category': 'u'},
  {'id': 3, 'name': 'ICEM', 'category': 'u'}],
 'links': []}

In [27]:
embed('@hoekr/group-in-a-box-layout', cells=['chart',  'style'])

In [28]:
embed("@ilogre/force-directed-graph", cells=['chart'], inputs={'data': jsondata})

In [29]:
import this
text = ''.join(this.d.get(l, l) for l in this.s)
embed('@d3/word-cloud', cells=['chart'], inputs={'source': text})

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [30]:
import csv, json
import sys, random #random for testing purposes
if sys.platform == 'win32':
    fl = open('/Users/rikhoekstra/Dropbox/act_doc.csv')
else:
    fl = open('/Users/rikhoekstra/Dropbox/testperdoc.csv')
ad = {}

df = csv.DictReader(fl)
for item in df:
    if ad.has_key(item['Perachternaam']):
        ad[item['Perachternaam']].append(item['ActDocumenten'])
    else:
        ad[item['Perachternaam']] = [item['ActDocumenten']]

class MakeNetwork(object):
    def __init__(self, d={}):
        self.nad = {}
        self.createStructure(d)

    def createStructure(self, ad={}):
        for k1, v1 in ad.iteritems():
            self.nad[k1] = [set([]), {}]
            for k2, v2 in ad.iteritems():
                if v1 and v2: #no empty sets in our structure
                    if k1 != k2 and len(set(v1).intersection(set(v2)))>0:
                        self.nad[k1][0].add(k2)
                        if k2 in self.nad[k1][1].keys():
                            print (self.nad[k1][1].keys())
                            self.nad[k1][1][k2] += 1
                        else:
                            self.nad[k1][1][k2] = 1

    def makeJson(self):
        jsontempl = []
        for item in self.nad.keys():
            jsontempl.append({'name': item, 'group': random.choice(xrange(3))})
        jsontempl2 = []
        for k, v in self.nad.iteritems():
            r = {}
            r['source'] = self.nad.keys().index(k)
            for i in v[0]:
                r['target'] = self.nad.keys().index(i)
#                import pdb; pdb.set_trace() 

                r['value'] = len(set(ad[k]).intersection(set(ad[i]))) #random.choice(xrange(10))#
            if r.has_key('target'):
                jsontempl2.append(r)
        linked_pers = {'nodes': jsontempl, 'links': jsontempl2}
        return linked_pers


    def writeJson(self, fl):
        fl = open(fl, 'w')
        json.dump(self.makeJson(), fl, indent=4)
        fl.close()

    def __call__(self):
        return self.makeJson()



FileNotFoundError: [Errno 2] No such file or directory: '/Users/rikhoekstra/Dropbox/testperdoc.csv'

In [31]:
from scripts.network_analysis import retrieve_spreadsheet_records

In [34]:
retrieve_spreadsheet_records("categories")

[{'organisation': 'REMP',
  'period_start': '1952',
  'last_known_date': '1983',
  'prs_id': '1',
  'prs_surname': 'Beijer',
  'prs_infix': '',
  'prs_initials': 'G.',
  'prs_function': 'demographer, The Hague',
  'prs_category': 'academic',
  'is_academic': 'yes',
  'is_public_administration': '',
  'Sources': '',
  'prs_country': 'NL',
  'prs_role1': 'founder',
  'prs_role2': 'member_MC',
  'prs_role3': 'secretary-editor',
  'remarks': 'director-editor (1969)'},
 {'organisation': 'REMP',
  'period_start': '1952',
  'last_known_date': '1969',
  'prs_id': '2',
  'prs_surname': 'Groenman',
  'prs_infix': '',
  'prs_initials': 'Sj.',
  'prs_function': 'sociologist, Leiden',
  'prs_category': 'academic',
  'is_academic': '1947',
  'is_public_administration': '1943-1950',
  'Sources': 'https://nl.wikipedia.org/wiki/Sjoerd_Groenman (specific version: https://nl.wikipedia.org/w/index.php?title=Sjoerd_Groenman&oldid=58778517)',
  'prs_country': 'NL',
  'prs_role1': 'founder',
  'prs_role2': '